In [43]:
import numpy as np
import random
import math
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set
from queue import Queue
import warnings
import sys
import json
import time
import types
import pickle
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import os

In [2]:
class Map(NamedTuple):
    grid: np.ndarray
    start: Union[Tuple[float, float], Tuple[float, float, float]]
    goal: Union[Tuple[float, float], Tuple[float, float, float]]
    obstacles: List[Union[Tuple[float, float, float, float], Tuple[float, float, float, float, float, float]]] # x, y, width, height or x, y, z, width, height, dimension
    size: Union[Tuple[int, int], Tuple[int, int, int]]

class PlannerResult(NamedTuple):
    success: bool
    path: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    nodes: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    edges: List[Tuple[Tuple[float, ...], Tuple[float, ...]]]  # (parent, child)

class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True  # 장애물 충돌 여부 등


In [ ]:
class MapGenerator:
    def __init__(
        self,
        map_type: Literal["random", "multi_narrow", "maze"] = "random",
        map_size: Union[Tuple[int, int], Tuple[int, int, int]] = (50, 50),
        obstacle_percent: float = 0.2,
        min_obstacle_size: Union[Tuple[int, int], Tuple[int, int, int]] = (2, 2),
        max_obstacle_size: Union[Tuple[int, int], Tuple[int, int, int]] = (5, 5),
        max_obstacle_count: Optional[int] = None
    ):
        self.map_type = map_type
        self.map_size = map_size
        self.obstacle_percent = obstacle_percent
        self.obstacle_list: List[Tuple[int, ...]] = []
        self.min_size = min_obstacle_size
        self.max_size = max_obstacle_size
        self.max_count = max_obstacle_count
        self.is_3d = len(map_size) == 3
        self.map = self._init_map()
        self.start = None
        self.goal = None

    def _init_map(self):
        shape = self.map_size[::-1] if self.is_3d else (self.map_size[1], self.map_size[0])
        return np.zeros(shape, dtype=np.uint8)

    def generate(self, start: Tuple[int, ...], goal: Tuple[int, ...]) -> Dict[str, Union[np.ndarray, Tuple[int, ...], List[Tuple[int, ...]], Tuple[int, int]]]:
        self.start = start
        self.goal = goal
        tries = 0
        max_tries = 100

        while tries < max_tries:
            self.map = self._init_map()
            self.obstacle_list.clear()

            if self.map_type == "maze":
                self.map = self._generate_maze()
            elif self.map_type == "random":
                self._generate_random_obstacles(start, goal)
            elif self.map_type == "multi_narrow":
                self._generate_multi_narrow(start, goal)

            if self._path_exists(start, goal):
                return Map(
                    grid=self.map,
                    start=start,
                    goal=goal,
                    obstacles=self.obstacle_list,
                    size=self.map_size
                )
            tries += 1

        raise RuntimeError("Failed to generate a connected map after multiple attempts.")

    def _add_obstacle(self, coords: Tuple[int, ...]):
        self.obstacle_list.append(coords)

    def _path_exists(self, start, goal):
        visited = set()
        q = Queue()
        q.put(start)
        visited.add(start)

        dims = len(start)
        neighbors = [(-1,0), (1,0), (0,-1), (0,1)] if dims == 2 else \
                    [(-1,0,0),(1,0,0),(0,-1,0),(0,1,0),(0,0,-1),(0,0,1)]

        while not q.empty():
            node = q.get()
            if node == goal:
                return True

            for delta in neighbors:
                neighbor = tuple(node[i] + delta[i] for i in range(dims))
                if self._in_bounds(neighbor) and neighbor not in visited:
                    if self._is_free(neighbor):
                        visited.add(neighbor)
                        q.put(neighbor)
        return False

    def _in_bounds(self, p):
        if self.is_3d:
            z, y, x = p
            return 0 <= z < self.map.shape[0] and 0 <= y < self.map.shape[1] and 0 <= x < self.map.shape[2]
        else:
            y, x = p
            return 0 <= y < self.map.shape[0] and 0 <= x < self.map.shape[1]

    def _is_free(self, p):
        if self.is_3d:
            z, y, x = p
            return self.map[z, y, x] == 0
        else:
            y, x = p
            return self.map[y, x] == 0

    def _is_inside(self, point, x, y, z, ow, oh, od):
        px, py, *pz = point
        if z is None:
            return x <= px < x + ow and y <= py < y + oh
        else:
            pz = pz[0]
            return x <= px < x + ow and y <= py < y + oh and z <= pz < z + od

    def _generate_random_obstacles(self, start, goal):
        if self.is_3d:
            w, h, d = self.map_size
            total_voxels = w * h * d
            max_obs = int(total_voxels * self.obstacle_percent)
            count = 0
            for _ in range(10000):
                ow = random.randint(self.min_size[0], self.max_size[0])
                oh = random.randint(self.min_size[1], self.max_size[1])
                od = random.randint(self.min_size[2], self.max_size[2])
                x = random.randint(0, w - ow - 1)
                y = random.randint(0, h - oh - 1)
                z = random.randint(0, d - od - 1)
                if np.any(self.map[z:z+od, y:y+oh, x:x+ow]):
                    continue
                if self._is_inside(start, x, y, z, ow, oh, od) or self._is_inside(goal, x, y, z, ow, oh, od):
                    continue
                self.map[z:z+od, y:y+oh, x:x+ow] = 1
                self._add_obstacle((x, y, z, ow, oh, od))
                count += ow * oh * od
                if self.max_count and count >= self.max_count:
                    break
                if count >= max_obs:
                    break
        else:
            w, h = self.map_size
            total_cells = w * h
            max_obs = int(total_cells * self.obstacle_percent)
            count = 0
            for _ in range(10000):
                ow = random.randint(self.min_size[0], self.max_size[0])
                oh = random.randint(self.min_size[1], self.max_size[1])
                x = random.randint(0, w - ow - 1)
                y = random.randint(0, h - oh - 1)
                if np.any(self.map[y:y+oh, x:x+ow]):
                    continue
                if self._is_inside(start, x, y, None, ow, oh, None) or self._is_inside(goal, x, y, None, ow, oh, None):
                    continue
                self.map[y:y+oh, x:x+ow] = 1
                self._add_obstacle((x, y, ow, oh))
                count += ow * oh
                if self.max_count and count >= self.max_count:
                    break
                if count >= max_obs:
                    break

    def _generate_multi_narrow(self, start, goal):
        if self.is_3d:
            self._generate_random_obstacles(start, goal)
        else:
            w, h = self.map_size
            corridor_width = 2
            spacing = 6
            for i in range(spacing, h - spacing, spacing + corridor_width):
                self.map[i:i+spacing, :] = 1
            self._carve_corridor(start, goal)

    def _carve_corridor(self, start, goal):
        if not self.is_3d:
            y0, x0 = start[1], start[0]
            y1, x1 = goal[1], goal[0]
            for x in range(min(x0, x1), max(x0, x1)+1):
                self.map[y0, x] = 0
            for y in range(min(y0, y1), max(y0, y1)+1):
                self.map[y, x1] = 0

    def _generate_maze(self):
        width, height = self.map_size
        if width % 2 == 0: width += 1
        if height % 2 == 0: height += 1

        maze = np.ones((height, width), dtype=np.uint8)
        sx, sy = 1, 1
        maze[sy, sx] = 0

        walls = [(sx + dx, sy + dy) for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]
                 if 0 < sx + dx < width and 0 < sy + dy < height]

        while walls:
            wx, wy = walls.pop(random.randint(0, len(walls) - 1))
            if maze[wy, wx] == 1:
                neighbors = [(wx + dx, wy + dy) for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]
                             if 0 < wx + dx < width and 0 < wy + dy < height and maze[wy + dy, wx + dx] == 0]
                if len(neighbors) == 1:
                    nx, ny = neighbors[0]
                    maze[(wy + ny) // 2, (wx + nx) // 2] = 0
                    maze[wy, wx] = 0
                    for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]:
                        nx, ny = wx + dx, wy + dy
                        if 0 < nx < width and 0 < ny < height and maze[ny, nx] == 1:
                            walls.append((nx, ny))

        return maze



In [3]:
class MapIO:
    @staticmethod
    def save_map(map_data: Map, filename: str) -> None:
        """Save Map object to a binary file."""
        with open(filename, 'wb') as f:
            pickle.dump(map_data, f)

    @staticmethod
    def load_map(filename: str) -> Map:
        """Load Map object from a binary file."""
        if not os.path.exists(filename):
            raise FileNotFoundError(f"Map file not found: {filename}")
        with open(filename, 'rb') as f:
            return pickle.load(f)

In [4]:
def visualize_map_shapes(
    map_array: np.ndarray,
    start: Optional[Tuple[int, ...]] = None,
    goal: Optional[Tuple[int, ...]] = None,
    obs:List[Tuple[int, ...]] = None,
    path: Optional[List[Tuple[float, ...]]] = None,
    nodes: Optional[List[Tuple[float, ...]]] = None,
    edges: Optional[List[Tuple[float, ...]]] = None,
    title: str = "Map Visualization"
):
    fig = go.Figure()

    if map_array.ndim == 2:
        height, width = map_array.shape
        
        for x, y, w, h in obs:
                fig.add_shape(
                    type="rect",
                    x0=x, x1=x+w, y0=y, y1=y+h,
                    fillcolor="purple",opacity=0.5,
                    line=dict(width=0)
                )

        # 방문 노드
        if nodes:
            vx, vy = zip(*nodes)
            fig.add_trace(go.Scatter(
                x=vx, y=vy, mode="markers",
                marker=dict(size=4, color="blue"),
                name="nodes"
            ))

        # 경로
        if path:
            px, py = zip(*path)
            fig.add_trace(go.Scatter(
                x=px, y=py, mode="lines+markers",
                line=dict(color="green"),
                marker=dict(size=6),
                name="Path"
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter(
                    x=[parent.position[0], child.position[0]], y=[parent.position[1], child.position[1]],
                    mode="lines",
                    line=dict(color="lightblue", width=1),
                    showlegend=False,
                    hoverinfo="skip"
                ))


        # 시작/목표
        if start:
            fig.add_trace(go.Scatter(
                x=[start[0]], y=[start[1]], mode="markers",
                marker=dict(size=10, color="red"),
                name="Start"
            ))

        if goal:
            fig.add_trace(go.Scatter(
                x=[goal[0]], y=[goal[1]], mode="markers",
                marker=dict(size=10, color="orange"),
                name="Goal"
            ))

        fig.add_shape(
            type="rect",
            x0=0, y0=0,
            x1=width, y1=height,
            line=dict(color="white", width=3),
            fillcolor="rgba(0,0,0,0)",  # 투명 내부
            layer="above"
        )

        fig.update_layout(
            title=title,
            xaxis=dict(scaleanchor="y", showgrid=False),
            # yaxis=dict(showgrid=False, autorange="reversed"),
            yaxis=dict(showgrid=False),
            height=600, width=600
        )

    elif map_array.ndim == 3:
        z, y, x = map_array.nonzero()
        x, y, z = list(x), list(y), list(z)

        # for x,y,w,h in obs:
        #     fig.add_trace(go.Mesh3d(
        #         x=x, y=y, z=z,
        #         color='black',
        #         opacity=1.0,
        #         alphahull=0,
        #         name='Obstacles'
        #     ))

        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            color='black',
            opacity=1.0,
            alphahull=0,
            name='Obstacles'
        ))

        if nodes:
            vx, vy, vz = zip(*nodes)
            fig.add_trace(go.Scatter3d(
                x=vx, y=vy, z=vz,
                mode='markers',
                marker=dict(size=2, color='blue'),
                name='Visited'
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter3d(
                    x=[parent.position[0], child.position[0]],
                    y=[parent[1].position, child.position[1]],
                    z=[parent[2].position, child.position[2]],
                    mode='lines',
                    line=dict(color='lightblue', width=2),
                    showlegend=False,
                    hoverinfo="skip"
                ))

        if path:
            px_, py_, pz_ = zip(*path)
            fig.add_trace(go.Scatter3d(
                x=px_, y=py_, z=pz_,
                mode='lines+markers',
                marker=dict(size=3, color='green'),
                name='Path'
            ))

        if start:
            fig.add_trace(go.Scatter3d(
                x=[start[0]], y=[start[1]], z=[start[2]],
                mode='markers',
                marker=dict(size=5, color='red'),
                name='Start'
            ))

        if goal:
            fig.add_trace(go.Scatter3d(
                x=[goal[0]], y=[goal[1]], z=[goal[2]],
                mode='markers',
                marker=dict(size=5, color='orange'),
                name='Goal'
            ))

        fig.update_layout(
            title=title,
            scene=dict(aspectmode='data'),
            height=700, width=700
        )

        

    fig.show()


In [ ]:
gen = MapGenerator(map_type="random", map_size=(100, 100), min_obstacle_size=(5, 5), max_obstacle_size=(15, 15), obstacle_percent=0.32)
m = gen.generate(start=(1, 1), goal=(99, 99))
visualize_map_shapes(m.grid, obs=m.obstacles, start=m.start, goal=m.goal)

In [ ]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (80,50), (50,50)
    

    grid[27:30, 52:70] = 1  
    obstacles.append((27, 52, 3, 18))
    grid[27:30, 30:48] = 1  
    obstacles.append((27, 30, 3, 18))
    
    grid[70:73, 30:70] = 1
    obstacles.append((70, 30, 3, 40))

    grid[30:70, 30:33] = 1
    obstacles.append((30, 30, 40, 3))
    grid[30:70, 67:70] = 1  
    obstacles.append((30, 67, 40, 3))


    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [ ]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (29,40), (90,10)
    


    grid[17:20, 20:80] = 1
    obstacles.append((17, 20, 3, 60))
    grid[37:40, 0:50] = 1
    obstacles.append((37, 0, 3, 50))
    grid[57:60, 20:100] = 1
    obstacles.append((57, 20, 3, 80))
    grid[77:80, 20:70] = 1
    obstacles.append((77, 0, 3, 80))

    grid[20:37, 47:60] = 1  
    obstacles.append((20, 47, 17, 3))
    grid[30:57, 62:65] = 1 
    obstacles.append((30, 62, 27, 3))
    grid[10:45, 80:83] = 1 
    obstacles.append((10, 80, 35, 3))
    grid[10:45, 80:83] = 1 
    obstacles.append((70, 80, 20, 3))


    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [112]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (29,10), (90,80)
    
    grid[27:30, 20:80] = 1
    obstacles.append((27, 20, 3, 60))
    grid[47:50, 0:50] = 1
    obstacles.append((47, 0, 3, 50))
    grid[77:80, 20:100] = 1
    obstacles.append((77, 20, 3, 80))

    grid[30:47, 47:50] = 1  
    obstacles.append((30, 47, 17, 3))
    grid[50:77, 72:65] = 1 
    obstacles.append((50, 72, 27, 3))

    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [113]:
map_obj = generate_box_with_narrow_entry(width=100, height=100, entry_side="left", entry_pos=25, entry_width=2)


In [114]:
visualize_map_shapes(map_obj.grid, obs=map_obj.obstacles, start=map_obj.start, goal=map_obj.goal)

In [ ]:
MapIO.save_map(map_obj, "Maze_map_easy.pkl")

In [5]:
maze_map = MapIO.load_map("Maze_map_easy.pkl")
print("Start:", maze_map.start)
print("Obstacles:", len(maze_map.obstacles))
print(maze_map.grid.shape)

Start: (29, 10)
Obstacles: 5
(100, 100)


In [ ]:
maze_map = MapIO.load_map("Maze_map.pkl")
print("Start:", maze_map.start)
print("Obstacles:", len(maze_map.obstacles))
print(maze_map.grid.shape)

In [6]:
narrow_map = MapIO.load_map("Narrow_map.pkl")
print("Start:", narrow_map.start)
print("Obstacles:", len(narrow_map.obstacles))
print(narrow_map.grid.shape)

Start: (80, 50)
Obstacles: 5
(100, 100)


In [7]:
# 불러오기
multi_obs_map = MapIO.load_map("Multi_obs_map.pkl")
print("Start:", multi_obs_map.start)
print("Obstacles:", len(multi_obs_map.obstacles))
print(multi_obs_map.grid.shape)

Start: (1, 1)
Obstacles: 35
(100, 100)


In [8]:
visualize_map_shapes(multi_obs_map.grid, obs=multi_obs_map.obstacles, start=multi_obs_map.start, goal=multi_obs_map.goal)

In [10]:
class MultiMapBenchmarker:
    def __init__(
        self,
        maps: List[np.ndarray],
        name: str = "Algorithm",
        iter = 10,
        seed = 42,
    ):
        self.set_seed(seed)

        self.maps = maps
        self.name = name
        self.results_df: pd.DataFrame = pd.DataFrame()
        self.iter = iter

    def run(self, algorithm) -> pd.DataFrame:
        results = []
        main_start_time = time.time()
        for i, map_ in enumerate(self.maps):
            for j in range(self.iter):
                start_time = time.time()
                try:
                    output = algorithm(map_)
                except Exception as e:
                    output = {"path": [], "nodes": [], "n_nodes": 0}
                end_time = time.time()

                print(f"Map {i+1}, Iteration {j+1}: Time taken: {end_time - start_time:.4f} seconds")

                path = output.path
                nodes = output.nodes
                num_nodes = len(nodes)
                success = output.success
                path_length = self._compute_path_length(path)

                results.append({
                    "map_id": i,
                    "iter" : j,
                    "algorithm": self.name,
                    "success": success,
                    "time_taken": end_time - start_time,
                    "num_nodes": num_nodes,
                    "path_length": path_length
                })

        print(f"Total time taken for all maps: {time.time() - main_start_time:.4f} seconds")

        self.results_df = pd.DataFrame(results)
        return self.results_df, self.get_avg()

    def _compute_path_length(self, path: List[Tuple[float, ...]]) -> float:
        if not path or len(path) < 2:
            return 0.0
        return sum(np.linalg.norm(np.array(path[i]) - np.array(path[i+1])) for i in range(len(path) - 1))

    def save_results(self, filename: str):
        if self.results_df.empty:
            raise RuntimeError("No results to save. Run benchmark first.")
        self.results_df.to_csv(filename, index=False)

    def plot_metrics(self, metric: str = "time_taken"):
        if self.results_df.empty:
            raise RuntimeError("No results to plot. Run benchmark first.")
        if metric not in self.results_df.columns:
            raise ValueError(f"Invalid metric: {metric}")

        fig = px.bar(
            self.results_df,
            x="map_id",
            y=metric,
            color="success",
            title=f"{self.name} - {metric} per map",
            labels={"map_id": "Map ID", metric: metric.replace('_', ' ').title()}
        )
        fig.show()

    def get_avg(self):
        if self.results_df.empty:
            raise RuntimeError("No results to save. Run benchmark first.")

        classic_summary = self.results_df.groupby('map_id').agg({
            'success': 'mean',           # 성공률
            'time_taken': 'mean',        # 평균 소요 시간
            'node_length': 'mean',      # 평균 노드 수
            'path_length': lambda x: x[x > 0].mean()  # path=0 제외한 평균 경로 길이
        }).rename(columns={
            'success': 'success_rate',
            'time_taken': 'time_avg',
            'node_length': 'node_length_avg',
            'path_length': 'path_length_avg'
        }).reset_index()
        return classic_summary
    
    @staticmethod
    def get_improvement(reference_result:pd.DataFrame, results:pd.DataFrame) -> pd.DataFrame:
        improvement_df = pd.DataFrame()
        
        improvement_df['success_improvement'] = (results['success_rate'] - reference_result['success_rate']) * 100 # percent point
        improvement_df['time_improvement'] = (results['time_avg'] - reference_result['time_avg']) / reference_result['success_rate'] * 100
        improvement_df['length_improvement'] = (results['path_length_avg'] - reference_result['path_length_avg']) / reference_result['success_rate'] * 100

        improvement_df['objective_score'] = (
            0.5 * improvement_df['success_improvement'] +
            0.3 * improvement_df['time_improvement'] +
            0.2 * improvement_df['length_improvement']
            )

        return improvement_df
    
    def set_seed(self, seed: int = 42):
        random.seed(seed)                    # Python random
        np.random.seed(seed)                 # NumPy

## Generate templete

In [ ]:
# --- PlannerResult structure ---
class PlannerResult(NamedTuple):
    success: bool                       # Path navigation success or not
    path: List[Tuple[float, ...]]       # Final path from start to goal
    nodes: List[Node]                   # All explored nodes
    edges: List[Tuple[Node, Node]]      # Parent-child connections


# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...] → 2D: (x,y), 3D: (x,y,z)
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost
        self.children = []
        self.valid = True               # For collision checking etc.

    #### Create additional methods if needed ####

# --- Main Planner ---
class Planner:
    def __init__(self, max_iter: int = 5000):
        self.max_iter = max_iter

    def plan(self, map: Map) -> PlannerResult:
        bounds = map.size                  # Tuple[int, ...]: (W,H) or (W,H,D)
        start_position = map.start         # Tuple[float, ...] (W,H) or (W,H,D)
        goal_position = map.goal           # Tuple[float, ...] (W,H) or (W,H,D)
        obstacles = map.obstacles          # Rectangular blocks: 2D=(x,y,w,h), 3D=(x,y,z,w,h,d)

        is_3d = len(bounds) == 3

        # Core data
        success_state = False # Path navigation success or not
        extracted_path: List[Tuple[float, ...]] = [] # Final path from start to goal
        nodes: List[Node] = [] # All explored nodes
        edges: List[Tuple[Node, Node]] = [] # Parent-child connections

        #### Place holder: path planning logic ####

        return PlannerResult(
            success=success_state,
            path=extracted_path,
            nodes=nodes,
            edges=edges
        )
    
    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

In [12]:
# Classic RRT
import random
import math
from typing import List, Tuple, Union, NamedTuple

# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...] → 2D: (x,y), 3D: (x,y,z)
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost
        self.children = []
        self.valid = True               # For collision checking etc.

    def distance(self, other: 'Node') -> float:
        return math.dist(self.position, other.position)


# --- Planner class implementing basic RRT ---
class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0, goal_sample_rate: float = 0.05):
        self.max_iter = max_iter
        self.step_size = step_size
        self.goal_sample_rate = goal_sample_rate

    def plan(self, map: 'Map') -> 'PlannerResult':
        bounds = map.size
        start_position = map.start
        goal_position = map.goal
        obstacles = map.obstacles

        is_3d = len(bounds) == 3
        dim = 3 if is_3d else 2

        root = Node(start_position)
        nodes: List[Node] = [root]

        for _ in range(self.max_iter):
            # 1. Sample random point
            if random.random() < self.goal_sample_rate:
                sample = goal_position
            else:
                sample = tuple(random.uniform(0, bounds[i]) for i in range(dim))

            # 2. Find nearest node
            nearest = min(nodes, key=lambda n: math.dist(n.position, sample))

            # 3. Steer
            new_position = self._steer(nearest.position, sample, self.step_size)
            new_node = Node(new_position, parent=nearest, cost=nearest.cost + math.dist(nearest.position, new_position))

            # 4. Collision check
            if not self._is_in_obstacle(new_position, obstacles, is_3d):
                nearest.children.append(new_node)
                nodes.append(new_node)

                # 5. Check goal reached
                if math.dist(new_position, goal_position) <= self.step_size:
                    goal_node = Node(goal_position, parent=new_node, cost=new_node.cost + math.dist(new_position, goal_position))
                    new_node.children.append(goal_node)
                    nodes.append(goal_node)

                    path = self._extract_path(goal_node)
                    edges = [(node.parent, node) for node in nodes if node.parent]
                    return PlannerResult(True, path, nodes, edges)

        # No valid path
        edges = [(node.parent, node) for node in nodes if node.parent]
        return PlannerResult(False, [], nodes, edges)

    def _steer(self, from_pos: Tuple[float, ...], to_pos: Tuple[float, ...], step_size: float) -> Tuple[float, ...]:
        vec = [t - f for f, t in zip(from_pos, to_pos)]
        dist = math.dist(from_pos, to_pos)
        if dist <= step_size:
            return to_pos
        scale = step_size / dist
        return tuple(f + scale * d for f, d in zip(from_pos, vec))

    def _is_in_obstacle(self, pos: Tuple[float, ...], obstacles: List[Tuple], is_3d: bool) -> bool:
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _extract_path(self, node: Node) -> List[Tuple[float, ...]]:
        path = []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]

In [153]:
# RRT_star
# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...]
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost from start
        self.children = []             # List[Node]
        self.valid = True              # For collision checking etc.

    def distance(self, other: 'Node') -> float:
        return math.dist(self.position, other.position)


# --- Planner class implementing RRT* ---
class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0, goal_sample_rate: float = 0.05, neighbor_radius: float = 5.0):
        self.max_iter = max_iter
        self.step_size = step_size
        self.goal_sample_rate = goal_sample_rate
        self.neighbor_radius = neighbor_radius

    def plan(self, map: 'Map') -> 'PlannerResult':
        bounds = map.size
        start_position = map.start
        goal_position = map.goal
        obstacles = map.obstacles

        is_3d = len(bounds) == 3
        dim = 3 if is_3d else 2

        root = Node(start_position)
        nodes: List[Node] = [root]

        for _ in range(self.max_iter):
            # 1. Sample
            if random.random() < self.goal_sample_rate:
                sample = goal_position
            else:
                sample = tuple(random.uniform(0, bounds[i]) for i in range(dim))

            # 2. Nearest
            nearest = min(nodes, key=lambda n: math.dist(n.position, sample))

            # 3. Steer
            new_pos = self._steer(nearest.position, sample, self.step_size)
            if self._is_in_obstacle(new_pos, obstacles, is_3d):
                continue

            # 4. Create node
            new_node = Node(new_pos)

            # 5. Find near neighbors
            neighbors = [n for n in nodes if math.dist(n.position, new_pos) <= self.neighbor_radius]

            # 6. Choose parent with lowest cost
            best_parent = nearest
            min_cost = nearest.cost + math.dist(nearest.position, new_pos)
            for neighbor in neighbors:
                temp_cost = neighbor.cost + math.dist(neighbor.position, new_pos)
                if temp_cost < min_cost and not self._is_edge_in_obstacle(neighbor.position, new_pos, obstacles, is_3d):
                    best_parent = neighbor
                    min_cost = temp_cost

            new_node.parent = best_parent
            new_node.cost = min_cost
            best_parent.children.append(new_node)
            nodes.append(new_node)

            # 7. Rewire neighbors
            for neighbor in neighbors:
                potential_cost = new_node.cost + math.dist(new_node.position, neighbor.position)
                if potential_cost < neighbor.cost and not self._is_edge_in_obstacle(new_node.position, neighbor.position, obstacles, is_3d):
                    if neighbor.parent:
                        neighbor.parent.children.remove(neighbor)
                    neighbor.parent = new_node
                    neighbor.cost = potential_cost
                    new_node.children.append(neighbor)

            # 8. Goal check
            if math.dist(new_node.position, goal_position) <= self.step_size:
                goal_node = Node(goal_position, parent=new_node, cost=new_node.cost + math.dist(new_node.position, goal_position))
                new_node.children.append(goal_node)
                nodes.append(goal_node)
                path = self._extract_path(goal_node)
                edges = [(n.parent, n) for n in nodes if n.parent]
                return PlannerResult(True, path, nodes, edges)

        # Failure
        edges = [(n.parent, n) for n in nodes if n.parent]
        return PlannerResult(False, [], nodes, edges)

    def _steer(self, from_pos, to_pos, step_size):
        vec = [t - f for f, t in zip(from_pos, to_pos)]
        dist = math.dist(from_pos, to_pos)
        if dist <= step_size:
            return to_pos
        scale = step_size / dist
        return tuple(f + scale * d for f, d in zip(from_pos, vec))

    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    # def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, steps=20):
    #     for i in range(steps + 1):
    #         interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
    #         if self._is_in_obstacle(interp, obstacles, is_3d):
    #             return True
    #     return False
    
    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))  # resolution 예: 0.5 또는 1.0
        print(f"Checking edge from {from_pos} to {to_pos} with {steps} steps")
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                print(f"Edge from {from_pos} to {to_pos} intersects with obstacle at {interp}")
                return True
        return False

    def _extract_path(self, node: Node) -> List[Tuple[float, ...]]:
        path = []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]


In [160]:
# RRT-Connect
import random
import math
from typing import List, Tuple

class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

    def distance(self, other: 'Node') -> float:
        return math.dist(self.position, other.position)

class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0):
        self.max_iter = max_iter
        self.step_size = step_size

    def plan(self, map: 'Map') -> 'PlannerResult':
        bounds = map.size
        start_position = map.start
        goal_position = map.goal
        obstacles = map.obstacles

        is_3d = len(bounds) == 3
        dim = 3 if is_3d else 2

        tree_a: List[Node] = [Node(start_position)]
        tree_b: List[Node] = [Node(goal_position)]

        for _ in range(self.max_iter):
            sample = tuple(random.uniform(0, bounds[i]) for i in range(dim))

            if self._extend(tree_a, sample, obstacles, is_3d):
                if self._connect(tree_b, tree_a[-1].position, obstacles, is_3d):
                    path_a = self._extract_path(tree_a[-1])
                    path_b = self._extract_path(tree_b[-1])
                    full_path = path_a + path_b[::-1]
                    nodes = tree_a + tree_b
                    edges = [(n.parent, n) for n in nodes if n.parent]
                    return PlannerResult(True, full_path, nodes, edges)

            tree_a, tree_b = tree_b, tree_a

        nodes = tree_a + tree_b
        edges = [(n.parent, n) for n in nodes if n.parent]
        return PlannerResult(False, [], nodes, edges)

    def _extend(self, tree: List[Node], target: Tuple[float, ...], obstacles, is_3d: bool) -> bool:
        nearest = min(tree, key=lambda n: math.dist(n.position, target))
        new_pos = self._steer(nearest.position, target, self.step_size)
        if not self._is_in_obstacle(new_pos, obstacles, is_3d):
            new_node = Node(new_pos, parent=nearest, cost=nearest.cost + math.dist(nearest.position, new_pos))
            nearest.children.append(new_node)
            tree.append(new_node)
            return True
        return False

    def _connect(self, tree: List[Node], target: Tuple[float, ...], obstacles, is_3d: bool) -> bool:
        while True:
            if not self._extend(tree, target, obstacles, is_3d):
                return False
            if math.dist(tree[-1].position, target) <= self.step_size:
                return True

    def _steer(self, from_pos: Tuple[float, ...], to_pos: Tuple[float, ...], step_size: float) -> Tuple[float, ...]:
        vec = [t - f for f, t in zip(from_pos, to_pos)]
        dist = math.dist(from_pos, to_pos)
        if dist <= step_size:
            return to_pos
        scale = step_size / dist
        return tuple(f + scale * d for f, d in zip(from_pos, vec))

    def _is_in_obstacle(self, pos: Tuple[float, ...], obstacles: List[Tuple], is_3d: bool) -> bool:
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _extract_path(self, node: Node) -> List[Tuple[float, ...]]:
        path = []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]

In [ ]:
# RRT*-connect
# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...] → 2D: (x,y), 3D: (x,y,z)
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost
        self.children = []
        self.valid = True               # For collision checking etc.


# --- Planner class implementing RRT*-Connect ---
class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0, goal_sample_rate: float = 0.05):
        self.max_iter = max_iter
        self.step_size = step_size
        self.goal_sample_rate = goal_sample_rate

    def plan(self, map: Map) -> PlannerResult:
        from math import dist
        import random

        bounds = map.size
        start_position = map.start
        goal_position = map.goal
        obstacles = map.obstacles

        is_3d = len(bounds) == 3
        dim = 3 if is_3d else 2

        tree_start: List[Node] = [Node(start_position)]
        tree_goal: List[Node] = [Node(goal_position)]

        for _ in range(self.max_iter):
            if random.random() < self.goal_sample_rate:
                sample = goal_position
            else:
                sample = tuple(random.uniform(0, bounds[i]) for i in range(dim))

            # Extend from start tree
            new_node_start = self._extend(tree_start, sample, obstacles, is_3d)
            if new_node_start is None:
                continue

            # Connect from goal tree to the new node
            new_node_goal = self._connect(tree_goal, new_node_start.position, obstacles, is_3d)

            if new_node_goal:
                # Trees connected, build path
                path_from_start = self._extract_path(new_node_start)
                path_from_goal = self._extract_path(new_node_goal)[::-1]
                full_path = path_from_start + path_from_goal
                all_nodes = tree_start + tree_goal
                edges = [(n.parent, n) for n in all_nodes if n.parent]
                return PlannerResult(True, full_path, all_nodes, edges)

            # Swap trees
            tree_start, tree_goal = tree_goal, tree_start

        all_nodes = tree_start + tree_goal
        edges = [(n.parent, n) for n in all_nodes if n.parent]
        return PlannerResult(False, [], all_nodes, edges)

    def _extend(self, tree: List[Node], sample: Tuple[float, ...], obstacles, is_3d: bool) -> Node:
        from math import dist

        nearest = min(tree, key=lambda n: dist(n.position, sample))
        new_pos = self._steer(nearest.position, sample, self.step_size)
        if self._is_in_obstacle(new_pos, obstacles, is_3d):
            return None

        new_node = Node(new_pos, parent=nearest, cost=nearest.cost + dist(nearest.position, new_pos))
        nearest.children.append(new_node)
        tree.append(new_node)

        # RRT* rewiring
        near_nodes = [n for n in tree if dist(n.position, new_node.position) <= self.step_size * 2]
        for neighbor in near_nodes:
            potential_cost = new_node.cost + dist(new_node.position, neighbor.position)
            if potential_cost < neighbor.cost and not self._is_in_obstacle(self._steer(new_node.position, neighbor.position, self.step_size), obstacles, is_3d):
                neighbor.parent = new_node
                neighbor.cost = potential_cost

        return new_node

    def _connect(self, tree: List[Node], target: Tuple[float, ...], obstacles, is_3d: bool) -> Node:
        from math import dist

        current = min(tree, key=lambda n: dist(n.position, target))
        while True:
            new_pos = self._steer(current.position, target, self.step_size)
            if self._is_in_obstacle(new_pos, obstacles, is_3d):
                return None
            new_node = Node(new_pos, parent=current, cost=current.cost + dist(current.position, new_pos))
            current.children.append(new_node)
            tree.append(new_node)

            if dist(new_pos, target) <= self.step_size:
                return new_node
            current = new_node

    def _steer(self, from_pos: Tuple[float, ...], to_pos: Tuple[float, ...], step: float) -> Tuple[float, ...]:
        from math import dist
        vec = [t - f for f, t in zip(from_pos, to_pos)]
        d = dist(from_pos, to_pos)
        if d <= step:
            return to_pos
        scale = step / d
        return tuple(f + scale * v for f, v in zip(from_pos, vec))

    def _is_in_obstacle(self, pos: Tuple[float, ...], obstacles, is_3d: bool) -> bool:
        if is_3d:
            for x, y, z, w, h, d in obstacles:
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
        else:
            for x, y, w, h in obstacles:
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _extract_path(self, node: Node) -> List[Tuple[float, ...]]:
        path = []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]


In [ ]:
# Bi-RRT
# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...]
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost
        self.children = []
        self.valid = True               # For collision checking etc.

    def distance(self, other: 'Node') -> float:
        return math.dist(self.position, other.position)

# --- Main Planner ---
class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0):
        self.max_iter = max_iter
        self.step_size = step_size

    def plan(self, map: 'Map') -> PlannerResult:
        bounds = map.size
        start_position = map.start
        goal_position = map.goal
        obstacles = map.obstacles

        is_3d = len(bounds) == 3
        dim = 3 if is_3d else 2

        tree_start: List[Node] = [Node(start_position)]
        tree_goal: List[Node] = [Node(goal_position)]

        for _ in range(self.max_iter):
            sample = tuple(random.uniform(0, bounds[i]) for i in range(dim))

            if self._try_extend(tree_start, sample, tree_goal, obstacles, is_3d):
                path = self._extract_path(tree_start[-1]) + self._extract_path(tree_goal[-1])[::-1]
                nodes = tree_start + tree_goal
                edges = [(node.parent, node) for node in nodes if node.parent]
                return PlannerResult(True, path, nodes, edges)

            # Swap trees for next iteration
            tree_start, tree_goal = tree_goal, tree_start

        nodes = tree_start + tree_goal
        edges = [(node.parent, node) for node in nodes if node.parent]
        return PlannerResult(False, [], nodes, edges)

    def _try_extend(self, tree: List[Node], sample: Tuple[float, ...], other_tree: List[Node], obstacles, is_3d) -> bool:
        nearest = min(tree, key=lambda n: math.dist(n.position, sample))
        new_position = self._steer(nearest.position, sample, self.step_size)

        if self._is_in_obstacle(new_position, obstacles, is_3d):
            return False

        new_node = Node(new_position, parent=nearest, cost=nearest.cost + math.dist(nearest.position, new_position))
        nearest.children.append(new_node)
        tree.append(new_node)

        # Try connecting to other tree
        other_nearest = min(other_tree, key=lambda n: math.dist(n.position, new_node.position))
        if math.dist(new_node.position, other_nearest.position) <= self.step_size:
            final_node = Node(other_nearest.position, parent=new_node, cost=new_node.cost + math.dist(new_node.position, other_nearest.position))
            new_node.children.append(final_node)
            tree.append(final_node)
            other_tree.append(other_nearest)  # Ensure it's captured if not already
            return True
        return False

    def _steer(self, from_pos: Tuple[float, ...], to_pos: Tuple[float, ...], step_size: float) -> Tuple[float, ...]:
        vec = [t - f for f, t in zip(from_pos, to_pos)]
        dist = math.dist(from_pos, to_pos)
        if dist <= step_size:
            return to_pos
        scale = step_size / dist
        return tuple(f + scale * d for f, d in zip(from_pos, vec))

    def _is_in_obstacle(self, pos: Tuple[float, ...], obstacles: List[Tuple], is_3d: bool) -> bool:
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _extract_path(self, node: Node) -> List[Tuple[float, ...]]:
        path = []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]

In [ ]:
# Bi-RRT*
# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

# --- Bi-RRT* Planner ---
class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0, radius: float = 5.0):
        self.max_iter = max_iter
        self.step_size = step_size
        self.radius = radius

    def plan(self, map: 'Map') -> PlannerResult:
        import random, math

        bounds = map.size
        start_position = map.start
        goal_position = map.goal
        obstacles = map.obstacles

        is_3d = len(bounds) == 3
        dim = 3 if is_3d else 2

        tree_start: List[Node] = [Node(start_position)]
        tree_goal: List[Node] = [Node(goal_position)]

        success = False
        connection_node = None

        for _ in range(self.max_iter):
            sample = tuple(random.uniform(0, bounds[i]) for i in range(dim))
            for tree_a, tree_b in [(tree_start, tree_goal), (tree_goal, tree_start)]:
                nearest = min(tree_a, key=lambda n: self._dist(n.position, sample))
                new_pos = self._steer(nearest.position, sample)
                if self._in_obstacle(new_pos, obstacles, is_3d):
                    continue
                new_node = Node(new_pos, parent=nearest, cost=nearest.cost + self._dist(nearest.position, new_pos))
                near_nodes = [n for n in tree_a if self._dist(n.position, new_pos) <= self.radius]
                best_parent = min(near_nodes, key=lambda n: n.cost + self._dist(n.position, new_pos), default=nearest)
                new_node.parent = best_parent
                new_node.cost = best_parent.cost + self._dist(best_parent.position, new_pos)
                best_parent.children.append(new_node)
                tree_a.append(new_node)

                for near in near_nodes:
                    new_cost = new_node.cost + self._dist(new_node.position, near.position)
                    if new_cost < near.cost:
                        near.parent = new_node
                        near.cost = new_cost

                # Try to connect the two trees
                connect_node = self._try_connect(new_node, tree_b, is_3d, obstacles)
                if connect_node:
                    success = True
                    connection_node = (new_node, connect_node)
                    break
            if success:
                break

        nodes = tree_start + tree_goal
        edges = [(n.parent, n) for n in nodes if n.parent]
        path = (
            self._extract_path(connection_node[0])[:-1]
            + self._extract_path(connection_node[1])[::-1]
            if success else []
        )

        return PlannerResult(success, path, nodes, edges)

    def _dist(self, a, b):
        import math
        return math.dist(a, b)

    def _steer(self, from_pos, to_pos):
        import math
        vec = [t - f for f, t in zip(from_pos, to_pos)]
        dist = math.dist(from_pos, to_pos)
        if dist <= self.step_size:
            return to_pos
        scale = self.step_size / dist
        return tuple(f + scale * v for f, v in zip(from_pos, vec))

    def _in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _try_connect(self, node, tree, is_3d, obstacles):
        for n in tree:
            if self._dist(n.position, node.position) <= self.step_size:
                if not self._in_obstacle(n.position, obstacles, is_3d):
                    return n
        return None

    def _extract_path(self, node: Node) -> List[Tuple[float, ...]]:
        path = []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]

In [ ]:
# Informed RRT*

class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

    def distance(self, other_pos) -> float:
        return math.dist(self.position, other_pos)

class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0):
        self.max_iter = max_iter
        self.step_size = step_size

    def plan(self, map: Map) -> PlannerResult:
        bounds = map.size
        start_pos = map.start
        goal_pos = map.goal
        obstacles = map.obstacles
        is_3d = len(bounds) == 3

        nodes: List[Node] = []
        edges: List[Tuple[Node, Node]] = []
        success_state = False
        extracted_path: List[Tuple[float, ...]] = []

        start_node = Node(start_pos)
        goal_node = Node(goal_pos)
        nodes.append(start_node)
        best_cost = float("inf")
        c_best = float("inf")

        for _ in range(self.max_iter):
            if c_best < float("inf"):
                sample = self._sample_in_ellipse(start_pos, goal_pos, c_best, bounds)
            else:
                sample = self._sample_uniform(bounds)

            nearest = self._nearest_node(nodes, sample)
            new_pos = self._steer(nearest.position, sample)
            if not self._collision(nearest.position, new_pos, obstacles, is_3d):
                new_node = Node(new_pos)
                near_nodes = self._find_near(nodes, new_node, radius=5.0)
                min_cost = nearest.cost + nearest.distance(new_pos)
                min_parent = nearest

                for near in near_nodes:
                    if not self._collision(near.position, new_pos, obstacles, is_3d):
                        cost = near.cost + near.distance(new_pos)
                        if cost < min_cost:
                            min_cost = cost
                            min_parent = near

                new_node.cost = min_cost
                new_node.parent = min_parent
                min_parent.children.append(new_node)
                nodes.append(new_node)
                edges.append((min_parent, new_node))

                for near in near_nodes:
                    if not self._collision(new_pos, near.position, obstacles, is_3d):
                        new_cost = new_node.cost + new_node.distance(near.position)
                        if new_cost < near.cost:
                            if near.parent:
                                near.parent.children.remove(near)
                            near.parent = new_node
                            new_node.children.append(near)
                            near.cost = new_cost
                            edges.append((new_node, near))

                if new_node.distance(goal_pos) <= self.step_size:
                    if not self._collision(new_node.position, goal_pos, obstacles, is_3d):
                        goal_node.parent = new_node
                        goal_node.cost = new_node.cost + new_node.distance(goal_pos)
                        new_node.children.append(goal_node)
                        nodes.append(goal_node)
                        edges.append((new_node, goal_node))
                        success_state = True
                        c_best = goal_node.cost
                        extracted_path = self._extract_path(goal_node)

        return PlannerResult(
            success=success_state,
            path=extracted_path,
            nodes=nodes,
            edges=edges
        )

    def _sample_uniform(self, bounds):
        return tuple(random.uniform(0, b) for b in bounds)

    def _sample_in_ellipse(self, start, goal, c_best, bounds):
        c_min = math.dist(start, goal)
        if c_min == 0:
            return self._sample_uniform(bounds)

        center = tuple((s + g) / 2 for s, g in zip(start, goal))
        a = c_best / 2
        b = math.sqrt(a**2 - (c_min / 2)**2)
        while True:
            x = random.uniform(-1, 1)
            y = random.uniform(-1, 1)
            if x**2 + y**2 <= 1:
                break
        sample = (
            b * x + center[0],
            a * y + center[1]
        )
        return self._clip_to_bounds(sample, bounds)

    def _clip_to_bounds(self, sample, bounds):
        return tuple(max(0, min(s, b)) for s, b in zip(sample, bounds))

    def _nearest_node(self, nodes, point):
        return min(nodes, key=lambda n: math.dist(n.position, point))

    def _steer(self, from_pos, to_pos):
        vec = tuple(t - f for f, t in zip(from_pos, to_pos))
        dist = math.dist(from_pos, to_pos)
        if dist <= self.step_size:
            return to_pos
        ratio = self.step_size / dist
        return tuple(f + ratio * (t - f) for f, t in zip(from_pos, to_pos))

    def _collision(self, from_pos, to_pos, obstacles, is_3d):
        steps = int(math.dist(from_pos, to_pos) / 0.5)
        if steps == 0:
            steps = 1
        for i in range(steps + 1):
            inter = tuple(f + (t - f) * i / steps for f, t in zip(from_pos, to_pos))
            if self._in_obstacle(inter, obstacles, is_3d):
                return True
        return False

    def _in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                if x <= pos[0] <= x + w and y <= pos[1] <= y + h and z <= pos[2] <= z + d:
                    return True
            else:
                x, y, w, h = obs
                if x <= pos[0] <= x + w and y <= pos[1] <= y + h:
                    return True
        return False

    def _find_near(self, nodes, new_node, radius):
        return [node for node in nodes if node.distance(new_node.position) <= radius]

    def _extract_path(self, goal_node):
        path = []
        current = goal_node
        while current:
            path.append(current.position)
            current = current.parent
        return path[::-1]

In [ ]:
# Irrt*connect
import math
import random
from typing import List, Tuple, NamedTuple, Union

class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

    def distance(self, other_pos):
        return math.dist(self.position, other_pos)


class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0):
        self.max_iter = max_iter
        self.step_size = step_size

    def plan(self, map: Map) -> PlannerResult:
        bounds = map.size
        start_pos = map.start
        goal_pos = map.goal
        obstacles = map.obstacles
        is_3d = len(bounds) == 3

        nodes = [Node(start_pos)]
        edges = []
        c_best = float("inf")
        solution_set = []

        for _ in range(self.max_iter):
            if c_best < float("inf"):
                sample = self._informed_sample(start_pos, goal_pos, c_best, bounds)
            else:
                sample = self._random_sample(bounds)

            nearest = self._nearest(nodes, sample)
            new_pos = self._steer(nearest.position, sample)

            if self._collision(nearest.position, new_pos, obstacles, is_3d):
                continue

            new_node = Node(new_pos, parent=nearest, cost=nearest.cost + nearest.distance(new_pos))
            nodes.append(new_node)
            nearest.children.append(new_node)
            edges.append((nearest, new_node))

            # Check if we reached the goal region
            if new_node.distance(goal_pos) < self.step_size:
                goal_node = Node(goal_pos, parent=new_node, cost=new_node.cost + new_node.distance(goal_pos))
                nodes.append(goal_node)
                new_node.children.append(goal_node)
                edges.append((new_node, goal_node))
                solution_set.append(goal_node)
                c_best = min(c_best, goal_node.cost)

        if not solution_set:
            return PlannerResult(False, [], nodes, edges)

        best_goal = min(solution_set, key=lambda n: n.cost)
        path = self._extract_path(best_goal)
        return PlannerResult(True, path, nodes, edges)

    def _extract_path(self, node):
        path = []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]

    def _random_sample(self, bounds):
        return tuple(random.uniform(0, b) for b in bounds)

    def _nearest(self, nodes, sample_pos):
        return min(nodes, key=lambda n: math.dist(n.position, sample_pos))

    def _steer(self, from_pos, to_pos):
        direction = [t - f for f, t in zip(from_pos, to_pos)]
        dist = math.dist(from_pos, to_pos)
        if dist == 0:
            return from_pos
        ratio = self.step_size / dist
        return tuple(f + d * ratio for f, d in zip(from_pos, direction))

    def _collision(self, from_pos, to_pos, obstacles, is_3d, resolution=0.5):
        dist = math.dist(from_pos, to_pos)
        steps = max(int(dist / resolution), 1)
        for i in range(steps + 1):
            intermediate = tuple(f + (t - f) * i / steps for f, t in zip(from_pos, to_pos))
            if self._is_in_obstacle(intermediate, obstacles, is_3d):
                return True
        return False

    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                if x <= pos[0] <= x + w and y <= pos[1] <= y + h and z <= pos[2] <= z + d:
                    return True
            else:
                x, y, w, h = obs
                if x <= pos[0] <= x + w and y <= pos[1] <= y + h:
                    return True
        return False

    def _informed_sample(self, start, goal, c_max, bounds):
        c_min = math.dist(start, goal)
        if c_max < c_min:
            return self._random_sample(bounds)

        # Construct rotation matrix
        a1 = [(g - s) / c_min for s, g in zip(start, goal)]
        C = self._orthonormal_basis(a1, len(bounds))

        # Sample in unit ball
        while True:
            if len(bounds) == 2:
                x_ball = self._sample_unit_ball_2d()
            else:
                x_ball = self._sample_unit_ball_3d()
            L = [c_max / 2.0] + [math.sqrt(c_max**2 - c_min**2) / 2.0] * (len(bounds) - 1)
            scale = tuple(l * x for l, x in zip(L, x_ball))
            rotated = self._matvec(C, scale)
            x_rand = tuple(c + s for c, s in zip([(s + g) / 2.0 for s, g in zip(start, goal)], rotated))

            # Clamp to bounds
            if all(0 <= x_rand[i] <= bounds[i] for i in range(len(bounds))):
                return x_rand

    def _orthonormal_basis(self, a1, dim):
        import numpy as np
        a1 = np.array(a1)
        I = np.eye(dim)
        if np.allclose(a1, I[:, 0]):
            return I
        v = a1 + np.sign(a1[0]) * np.linalg.norm(a1) * I[:, 0]
        v /= np.linalg.norm(v)
        H = I - 2.0 * np.outer(v, v)
        return H @ I

    def _matvec(self, mat, vec):
        return tuple(sum(m * v for m, v in zip(row, vec)) for row in mat)

    def _sample_unit_ball_2d(self):
        r = math.sqrt(random.random())
        theta = random.uniform(0, 2 * math.pi)
        return (r * math.cos(theta), r * math.sin(theta))

    def _sample_unit_ball_3d(self):
        while True:
            x, y, z = random.uniform(-1, 1), random.uniform(-1, 1), random.uniform(-1, 1)
            if x**2 + y**2 + z**2 <= 1:
                return (x, y, z)

In [14]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method.json'

In [ ]:
result = {
    "algorithm": "Informed-RRT*",
    "algorithm_description": "Informed RRT* is an optimal sampling-based motion planning algorithm that improves the efficiency of RRT* by focusing the search space. Once a feasible solution is found, it restricts sampling to a prolate hyperspheroid (ellipsoid in 2D/3D) defined by the start and goal states and the cost of the current best solution. This results in faster convergence toward an optimal solution.",
    "planning_mechanism": '''
1.	Initialization: Start with the root node at the start position.
2.	Sampling:
	•	Before a solution is found: sample uniformly within the search space.
	•	After a solution is found: sample within the informed ellipsoid (heuristic-based subset).
3.	Nearest Node: Find the node in the tree closest to the sampled point.
4.	Steering: Move toward the sample, limited by a step size.
5.	Collision Check: Ensure the new edge is obstacle-free.
6.	Add Node: Add the new node to the tree.
7.	Rewire: Improve nearby nodes’ paths by rewiring through the new node if it reduces cost.
8.	Update Best Path: If the new node reaches the goal and has a lower cost, update the best solution.
9.	Repeat until iteration or time limit is reached.
''',
    "code": '''
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

    def distance(self, other_pos) -> float:
        return math.dist(self.position, other_pos)

class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0):
        self.max_iter = max_iter
        self.step_size = step_size

    def plan(self, map: Map) -> PlannerResult:
        bounds = map.size
        start_pos = map.start
        goal_pos = map.goal
        obstacles = map.obstacles
        is_3d = len(bounds) == 3

        nodes: List[Node] = []
        edges: List[Tuple[Node, Node]] = []
        success_state = False
        extracted_path: List[Tuple[float, ...]] = []

        start_node = Node(start_pos)
        goal_node = Node(goal_pos)
        nodes.append(start_node)
        best_cost = float("inf")
        c_best = float("inf")

        for _ in range(self.max_iter):
            if c_best < float("inf"):
                sample = self._sample_in_ellipse(start_pos, goal_pos, c_best, bounds)
            else:
                sample = self._sample_uniform(bounds)

            nearest = self._nearest_node(nodes, sample)
            new_pos = self._steer(nearest.position, sample)
            if not self._collision(nearest.position, new_pos, obstacles, is_3d):
                new_node = Node(new_pos)
                near_nodes = self._find_near(nodes, new_node, radius=5.0)
                min_cost = nearest.cost + nearest.distance(new_pos)
                min_parent = nearest

                for near in near_nodes:
                    if not self._collision(near.position, new_pos, obstacles, is_3d):
                        cost = near.cost + near.distance(new_pos)
                        if cost < min_cost:
                            min_cost = cost
                            min_parent = near

                new_node.cost = min_cost
                new_node.parent = min_parent
                min_parent.children.append(new_node)
                nodes.append(new_node)
                edges.append((min_parent, new_node))

                for near in near_nodes:
                    if not self._collision(new_pos, near.position, obstacles, is_3d):
                        new_cost = new_node.cost + new_node.distance(near.position)
                        if new_cost < near.cost:
                            if near.parent:
                                near.parent.children.remove(near)
                            near.parent = new_node
                            new_node.children.append(near)
                            near.cost = new_cost
                            edges.append((new_node, near))

                if new_node.distance(goal_pos) <= self.step_size:
                    if not self._collision(new_node.position, goal_pos, obstacles, is_3d):
                        goal_node.parent = new_node
                        goal_node.cost = new_node.cost + new_node.distance(goal_pos)
                        new_node.children.append(goal_node)
                        nodes.append(goal_node)
                        edges.append((new_node, goal_node))
                        success_state = True
                        c_best = goal_node.cost
                        extracted_path = self._extract_path(goal_node)

        return PlannerResult(
            success=success_state,
            path=extracted_path,
            nodes=nodes,
            edges=edges
        )

    def _sample_uniform(self, bounds):
        return tuple(random.uniform(0, b) for b in bounds)

    def _sample_in_ellipse(self, start, goal, c_best, bounds):
        c_min = math.dist(start, goal)
        if c_min == 0:
            return self._sample_uniform(bounds)

        center = tuple((s + g) / 2 for s, g in zip(start, goal))
        a = c_best / 2
        b = math.sqrt(a**2 - (c_min / 2)**2)
        while True:
            x = random.uniform(-1, 1)
            y = random.uniform(-1, 1)
            if x**2 + y**2 <= 1:
                break
        sample = (
            b * x + center[0],
            a * y + center[1]
        )
        return self._clip_to_bounds(sample, bounds)

    def _clip_to_bounds(self, sample, bounds):
        return tuple(max(0, min(s, b)) for s, b in zip(sample, bounds))

    def _nearest_node(self, nodes, point):
        return min(nodes, key=lambda n: math.dist(n.position, point))

    def _steer(self, from_pos, to_pos):
        vec = tuple(t - f for f, t in zip(from_pos, to_pos))
        dist = math.dist(from_pos, to_pos)
        if dist <= self.step_size:
            return to_pos
        ratio = self.step_size / dist
        return tuple(f + ratio * (t - f) for f, t in zip(from_pos, to_pos))

    def _collision(self, from_pos, to_pos, obstacles, is_3d):
        steps = int(math.dist(from_pos, to_pos) / 0.5)
        if steps == 0:
            steps = 1
        for i in range(steps + 1):
            inter = tuple(f + (t - f) * i / steps for f, t in zip(from_pos, to_pos))
            if self._in_obstacle(inter, obstacles, is_3d):
                return True
        return False

    def _in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                if x <= pos[0] <= x + w and y <= pos[1] <= y + h and z <= pos[2] <= z + d:
                    return True
            else:
                x, y, w, h = obs
                if x <= pos[0] <= x + w and y <= pos[1] <= y + h:
                    return True
        return False

    def _find_near(self, nodes, new_node, radius):
        return [node for node in nodes if node.distance(new_node.position) <= radius]

    def _extract_path(self, goal_node):
        path = []
        current = goal_node
        while current:
            path.append(current.position)
            current = current.parent
        return path[::-1]
    '''
}

# 저장
with open(json_path, "a") as f:
    json.dump(result, f, indent=4)

In [175]:
with open(json_path, "r") as f:
    result_data = json.load(f)

print(result_data[0].keys())

dict_keys(['algorithm', 'algorithm_description', 'planning_mechanism', 'code'])


In [24]:
import_string = '''
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set, TYPE_CHECKING, Type
import time
from queue import Queue
import numpy as np
import random
import math
import sys
import os

from eoh.problems.optimization.classic_benchmark_path_planning.utils.architecture_utils import PlannerResult, Map
'''

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    planning_module = types.ModuleType("planning_module")
    exec(import_string+result_data[0]['code'], planning_module.__dict__)
    sys.modules[planning_module.__name__] = planning_module

In [161]:
p = Planner(max_iter=5000, step_size=10)
# p = planning_module.Planner()

In [163]:
result = p.plan(map=multi_obs_map)

In [164]:
result2 = p.plan(map=maze_map)

In [165]:
result3 = p.plan(map=narrow_map)

In [166]:
visualize_map_shapes(multi_obs_map.grid, obs=multi_obs_map.obstacles, start=multi_obs_map.start, goal=multi_obs_map.goal,
                      path=result.path, nodes=list(map(lambda x: x.position, result.nodes)), edges=result.edges)

In [167]:
visualize_map_shapes(maze_map.grid, obs=maze_map.obstacles, start=maze_map.start, goal=maze_map.goal, path=result2.path, nodes=list(map(lambda x: x.position, result2.nodes)), edges=result2.edges)

In [168]:
visualize_map_shapes(narrow_map.grid, obs=narrow_map.obstacles, start=narrow_map.start, goal=narrow_map.goal, path=result3.path, nodes=list(map(lambda x: x.position, result3.nodes)), edges=result3.edges)

In [ ]:
def my_dummy_planner(map_array, start, goal):
    return {
        "path": [start, goal],
        "visited": [start],
        "nodes": 2
    }

bench = MultiMapBenchmarker(
    maps=[multi_obs_map, maze_map, narrow_map],
    algorithm=p.plan,
    name="MyRRT"
)


df = bench.run()
print(df)

In [ ]:
df2 = bench.get_avg()
print(df2)

In [29]:
p = Planner(max_iter=10000)

bench2 = MultiMapBenchmarker(
    maps=[multi_obs_map, maze_map, narrow_map],
    algorithm=p.plan,
    name="MyRRT"
)

In [30]:
df3 = bench2.run()

In [31]:
df4 = bench2.get_avg()

In [34]:
df4

,map_id,success_rate,time_avg,path_length_avg
0,0,1.0,0.113167,178.205710
1,1,1.0,1.821893,310.468351
2,2,1.0,0.617888,142.911073


In [37]:
df5 = MultiMapBenchmarker.get_improvement(df4, df4)

In [38]:
pd.concat([df4, df5], axis=1)

,map_id,success_rate,time_avg,path_length_avg,success_improvement,time_improvement,length_improvement,objective_score
0,0,1.0,0.113167,178.205710,0.0,0.0,0.0,0.0
1,1,1.0,1.821893,310.468351,0.0,0.0,0.0,0.0
2,2,1.0,0.617888,142.911073,0.0,0.0,0.0,0.0


In [35]:
MultiMapBenchmarker.get_improvement(df4, df4)['objective_score']

0    0.0
1    0.0
2    0.0
Name: objective_score, dtype: float64

In [ ]:
from eoh.llm.interface_LLM import InterfaceLLM
from eoh.utils.getParas import Paras
from eoh.methods.eoh.eoh_evolution import Evolution
from eoh.problems.optimization.classic_benchmark_path_planning.utils.prompts import GetPrompts
import re
paras = Paras()


In [170]:
ee = Evolution("api.openai.com", api_key, "gpt-4.1-mini-2025-04-14", False, None, True, GetPrompts())
illm = InterfaceLLM("api.openai.com", api_key, "gpt-4.1-mini-2025-04-14", False, None, True)

- check LLM API
remote llm api is used ...
- check LLM API
remote llm api is used ...


In [171]:
result_data[3]

{'algorithm': 'RRT*-Connect',
 'algorithm_description': 'RRT*-Connect is a hybrid sampling-based motion planning algorithm that merges the strengths of RRT* and RRT-Connect. It grows two trees from the start and goal positions, extending them toward randomly sampled points. At each iteration, it attempts to connect the two trees. Additionally, it performs rewiring to improve path quality, as in RRT*, leading to an asymptotically optimal solution.',
 'planning mechanism': '\n1.\tInitialization\n\t•\tInitialize two trees: one rooted at the start and the other at the goal.\n\t•\tInsert the root nodes (start and goal) into their respective trees.\n2.\tSampling\n\t•\tGenerate a random sample from the configuration space.\n\t•\tWith a small probability (goal bias), use the opposite tree’s root as the sample.\n3.\tExtension\n\t•\tFor the active tree, find the nearest node to the sample.\n\t•\tSteer toward the sample up to a fixed step size and create a new node.\n\t•\tIf the new node is colli

In [176]:
pc = ee.get_prompt([result_data[3]], 'e1')

In [177]:
print(pc)

Your task is to design and implement Path planning algorithm. Main objective is to improve the path planning performance below.
Reference Implementation:
Algorithm description: RRT*-Connect is a hybrid sampling-based motion planning algorithm that merges the strengths of RRT* and RRT-Connect. It grows two trees from the start and goal positions, extending them toward randomly sampled points. At each iteration, it attempts to connect the two trees. Additionally, it performs rewiring to improve path quality, as in RRT*, leading to an asymptotically optimal solution.
Planning Mechanism:

1.Initialization
	•	Initialize two trees: one rooted at the start and the other at the goal.
	•	Insert the root nodes (start and goal) into their respective trees.
2.Sampling
	•	Generate a random sample from the configuration space.
	•	With a small probability (goal bias), use the opposite tree’s root as the sample.
3.Extension
	•	For the active tree, find the nearest node to the sample.
	•	Steer toward t

In [8]:
gcode = illm.get_response(pc)

In [9]:
print(gcode)

{This algorithm implements a Bidirectional Informed RRT* planner with heuristic-based informed sampling and path smoothing. It grows two trees from start and goal states, focusing samples in an ellipsoidal informed set to concentrate the search on promising regions, improving efficiency and solution quality. It applies rewiring to optimize paths and uses a post-processing smoothing step for path smoothness and shorter paths.}  
(A random sample is drawn within an ellipsoid defined by the current best path cost to bias sampling, then nodes are extended towards samples from both trees. The two trees attempt to connect after each iteration, enabling quick discovery of a path. Rewiring improves local paths iteratively. When a path is found, path smoothing refines it by shortcutting unnecessary waypoints.)

```python
from typing import List, Tuple, NamedTuple, Optional
from math import dist, sqrt, cos, sin, acos
import random

# --- Node class ---
class Node:
    def __init__(self, position

In [11]:
c, m, a = ee._extract_alg(gcode)

In [16]:
print(a)

class Node:
    def __init__(self, position: Tuple[float, ...], parent: Optional['Node'] = None, cost: float = 0.0):
        self.position = position        # Tuple[float, ...] for 2D or 3D
        self.parent = parent            # Node or None
        self.cost = cost                # Cost from start node
        self.children: List[Node] = []
        self.valid = True               # For collision checking etc.

    def add_child(self, child: 'Node'):
        self.children.append(child)

    def path(self) -> List[Tuple[float, ...]]:
        node, p = self, []
        while node:
            p.append(node.position)
            node = node.parent
        return p[::-1]
class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 1.0, goal_sample_rate: float = 0.05, rewire_radius: float = 5.0):
        self.max_iter = max_iter
        self.step_size = step_size
        self.goal_sample_rate = goal_sample_rate
        self.rewire_radius = rewire_radius

    def plan(se